In [ ]:
# Información sobre las librerías utilizadas

https://colab.research.google.com/drive/1XCwu7NaCYXf3kkhlx4Iei4w2KrXzr1zo?usp=sharing

https://docs.sdv.dev/sdv/single-table-data/data-preparation/single-table-metadata-api

# Importamos librerías

In [ ]:
import pandas as pd

## Cargamos los datos

In [ ]:
df_inicial = pd.read_excel('C:/Users/amidonga/Documents/TFG/dfFalso.xlsx')

In [ ]:
df_inicial

# Contrucción de datos sintéticos

## Synthesizer
Un SDV **synthesizer** es un objeto utilizado para crear datos sintéticos. Utiliza una cópula para imitar los patrones de los datos reales y generar datos sintéticos. 

### Importamos las librerías necesarias

In [ ]:
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import SingleTableMetadata

### Ajustamos los tipos de datos

In [ ]:
metadata = {
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1",
    "primary_key": None,
    "columns": {
        "Comunidad Autónoma": {
            "sdtype": "categorical"
        },
        "Edad": {
            "sdtype": "numerical",
            "computer_representation": "Int"
        },
        "Procedencia": {
            "sdtype": "categorical"
        },
        "Sexo": {
            "sdtype": "categorical"
        },
        "Situación de dependencia": {
            "sdtype": "categorical"
        },
        "Sector económico": {
            "sdtype": "categorical"
        },
        "Renta anual neta": {
            "sdtype": "numerical",
            "computer_representation": "Float"
        },
        "Estado civil": {
            "sdtype": "categorical"
        },
        "Número de hijos": {
            "sdtype": "numerical",
            "computer_representation": "Int"
        }
    }
}


metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df_inicial)

synthesizer = GaussianCopulaSynthesizer(metadata)
synthesizer.fit(df_inicial)
synthetic_data = synthesizer.sample(num_rows= 100000)
synthetic_data.head()

### Definimos criterios de perturbación para evitar errores

In [ ]:
synthetic_data.loc[synthetic_data['Edad'] >= 65, 'Sector económico'] = 'Pensionista'
synthetic_data.loc[synthetic_data['Edad'] <= 14, 'Sector económico'] = 'Estudiante'
synthetic_data.loc[synthetic_data['Edad'] <= 17, 'Estado civil'] = 'Solteros/Solteras'
synthetic_data.loc[synthetic_data['Edad'] <= 14, 'Número de hijos'] = 0
synthetic_data.loc[((synthetic_data['Edad'] <= 20) &(synthetic_data['Número de hijos'] >1)), 'Número de hijos'] = 0
synthetic_data.loc[synthetic_data['Edad'] <= 14, 'Renta anual neta'] = 0

## Comparación de los datos sintéticos y los datos reales

In [ ]:
from sdv.evaluation.single_table import evaluate_quality

In [ ]:
quality_report = evaluate_quality(
    df_inicial,
    synthetic_data,
    metadata
)


El reporte nos permite visualizar las diferentes propiedades de los datos que la cópula ha logrado capturar. La siguiente visualización nos muestra qué columnas han conseguido capturarse mejor.

In [ ]:
quality_report.get_visualization('Column Shapes')

## Visualización de los datos
Visualizamos los datos reales y sintéticos enfrentados entre sí.


In [ ]:
from sdv.evaluation.single_table import get_column_plot
import matplotlib.pyplot as plt

In [ ]:
fig = get_column_plot(
    real_data=df_inicial,
    synthetic_data=synthetic_data,
    column_name='Sexo',
    metadata=metadata
)

fig.show()

In [ ]:
fig = get_column_plot(
    real_data=df_inicial,
    synthetic_data=synthetic_data,
    column_name='Comunidad Autónoma',
    metadata=metadata
)
    
fig.show()

In [ ]:
fig = get_column_plot(
    real_data=df_inicial,
    synthetic_data=synthetic_data,
    column_name='Edad',
    metadata=metadata
)
    
fig.show()

In [ ]:
fig = get_column_plot(
    real_data=df_inicial,
    synthetic_data=synthetic_data,
    column_name='Situación de dependencia',
    metadata=metadata
)
    
fig.show()

In [ ]:
fig = get_column_plot(
    real_data=df_inicial,
    synthetic_data=synthetic_data,
    column_name='Renta anual neta',
    metadata=metadata
)
    
fig.show()

In [ ]:
fig = get_column_plot(
    real_data=df_inicial,
    synthetic_data=synthetic_data,
    column_name='Estado civil',
    metadata=metadata
)
    
fig.show()

In [ ]:
fig = get_column_plot(
    real_data=df_inicial,
    synthetic_data=synthetic_data,
    column_name='Número de hijos',
    metadata=metadata
)
    
fig.show()

In [ ]:
fig = get_column_plot(
    real_data=df_inicial,
    synthetic_data=synthetic_data,
    column_name='Procedencia',
    metadata=metadata
)
    
fig.show()

In [ ]:
fig = get_column_plot(
    real_data=df_inicial,
    synthetic_data=synthetic_data,
    column_name='Sector económico',
    metadata=metadata
)
    
fig.show()

In [ ]:
from sdv.evaluation.single_table import get_column_pair_plot

In [ ]:
fig = get_column_pair_plot(
    real_data=df_inicial,
    synthetic_data=synthetic_data,
    column_names=['Número de hijos','Edad'],
    metadata=metadata
)
    
fig.show()

In [ ]:
fig = get_column_pair_plot(
    real_data=df_inicial,
    synthetic_data=synthetic_data,
    column_names=['Renta anual neta','Edad'],
    metadata=metadata
)
    
fig.show()

In [ ]:
fig = get_column_pair_plot(
    real_data=df_inicial,
    synthetic_data=synthetic_data,
    column_names=['Renta anual neta','Sector económico'],
    metadata=metadata
)
    
fig.show()

## 1.6 Saving and Loading
We can save the synthesizer to share with others and sample more synthetic data in the future.

In [ ]:
synthesizer.save('my_synthesizer.pkl')

# synthesizer = GaussianCopulaSynthesizer.load('my_synthesizer.pkl')
# synthetic_data.to_excel('C:/Users/amidonga/Documents/TFG/Datos_Sinteticos.xlsx',index=False)

# Asignación de los trámites a recomendar

Beca para la matrícula universitaria. Ofreceremos esta ayuda a personas que tienen entre 18 y 15 años de edad, con una renta inferior al percentil 75 de la variable "Renta anual neta" y que aparecen en "Sector económico" como estudiantes. 

Beca para formación en idiomas. Ofreceremos esta ayuda a personas de edades superiores a los 14 años. 

Beca para la transformación digital. Ofreceremos esta ayuda a personas con más de 55 años de edad. 

Beca para el transporte. Ofreceremos esta ayuda a personas menores de 30 años. 

Ayuda al desempleo. Ofreceremos esta ayuda a las personas que en la variable "Sector económico" aparecen como parados. 

Ayuda para incentivar el consumo. Ofreceremos esta ayuda a personas mayores de 20 años. 

Ayuda para personas en que viven en zonas rurales. Ofreceremos esta ayuda a personas mayores de 20 años en comunidades autónomas distintas a la Comunidad de Madrid, Cataluña, País Vasco y la Comunidad Valenciana. 


## T.1. Beca para la matrícula universitaria

In [ ]:
# Calcular el percentil 75 de la variable 'Renta anual neta' para obtener el umbral de renta:
renta_threshold = synthetic_data['Renta anual neta'].quantile(0.75)

#Filtrar el dataframe para obtener los registros que cumplen las condiciones especificadas:
filtro_edad = (synthetic_data['Edad'] >= 18) & (synthetic_data['Edad'] <= 25)
filtro_renta = synthetic_data['Renta anual neta'] < renta_threshold
filtro_sector = synthetic_data['Sector económico'] == 'Estudiante'
registros_cumplen_normas = synthetic_data[filtro_edad & filtro_renta & filtro_sector]

#Definir los porcentajes x, y, y z que sumen 100 y representen la proporción de registros que
# tendrán los valores 1, 0 y nulo respectivamente:
x = 75  # Porcentaje de registros que tendrán el valor 1
y = 20  # Porcentaje de registros que tendrán el valor 0
z = 100 - x - y  # Porcentaje de registros que tendrán el valor nulo


# Obtener una muestra aleatoria de los registros que cumplen las normas para asignarles el valor 1:
num_registros = len(registros_cumplen_normas)
num_x = int(num_registros * x / 100)
num_y = int(num_registros * y / 100)

# Crear la columna 'T.1.' y asignar los valores correspondientes:
synthetic_data['T.1.'] = None  # Inicializar la columna con valores nulos

# Asignar el valor 1 a un porcentaje x de registros aleatorios
registros_x = registros_cumplen_normas.sample(num_x)
synthetic_data.loc[registros_x.index, 'T.1.'] = 1

# Asignar el valor 0 a un porcentaje y de registros aleatorios
registros_y = registros_cumplen_normas.drop(registros_x.index).sample(num_y)
synthetic_data.loc[registros_y.index, 'T.1.'] = 0



## T.2. Beca para formación en idiomas

In [ ]:
#Filtrar el dataframe para obtener los registros que cumplen las condiciones especificadas:
filtro_edad = (synthetic_data['Edad'] >= 14)
registros_cumplen_normas = synthetic_data[filtro_edad]

#Definir los porcentajes x, y, y z que sumen 100 y representen la proporción de registros que
# tendrán los valores 1, 0 y nulo respectivamente:
x = 55  # Porcentaje de registros que tendrán el valor 1
y = 30  # Porcentaje de registros que tendrán el valor 0
z = 100 - x - y  # Porcentaje de registros que tendrán el valor nulo


# Obtener una muestra aleatoria de los registros que cumplen las normas para asignarles el valor 1:
num_registros = len(registros_cumplen_normas)
num_x = int(num_registros * x / 100)
num_y = int(num_registros * y / 100)

# Crear la columna 'T.1.' y asignar los valores correspondientes:
synthetic_data['T.2.'] = None  # Inicializar la columna con valores nulos

# Asignar el valor 1 a un porcentaje x de registros aleatorios
registros_x = registros_cumplen_normas.sample(num_x)
synthetic_data.loc[registros_x.index, 'T.2.'] = 1

# Asignar el valor 0 a un porcentaje y de registros aleatorios
registros_y = registros_cumplen_normas.drop(registros_x.index).sample(num_y)
synthetic_data.loc[registros_y.index, 'T.2.'] = 0

## T.3. Beca para la transformación digital

In [ ]:
#Filtrar el dataframe para obtener los registros que cumplen las condiciones especificadas:
filtro_edad = (synthetic_data['Edad'] >= 55) 
registros_cumplen_normas = synthetic_data[filtro_edad]

#Definir los porcentajes x, y, y z que sumen 100 y representen la proporción de registros que
# tendrán los valores 1, 0 y nulo respectivamente:
x = 50  # Porcentaje de registros que tendrán el valor 1
y = 35  # Porcentaje de registros que tendrán el valor 0
z = 100 - x - y  # Porcentaje de registros que tendrán el valor nulo


# Obtener una muestra aleatoria de los registros que cumplen las normas para asignarles el valor 1:
num_registros = len(registros_cumplen_normas)
num_x = int(num_registros * x / 100)
num_y = int(num_registros * y / 100)

# Crear la columna 'T.1.' y asignar los valores correspondientes:
synthetic_data['T.3.'] = None  # Inicializar la columna con valores nulos

# Asignar el valor 1 a un porcentaje x de registros aleatorios
registros_x = registros_cumplen_normas.sample(num_x)
synthetic_data.loc[registros_x.index, 'T.3.'] = 1

# Asignar el valor 0 a un porcentaje y de registros aleatorios
registros_y = registros_cumplen_normas.drop(registros_x.index).sample(num_y)
synthetic_data.loc[registros_y.index, 'T.3.'] = 0

## T.4. Beca para el transporte

In [ ]:
#Filtrar el dataframe para obtener los registros que cumplen las condiciones especificadas:
filtro_edad = (synthetic_data['Edad'] < 30) 
registros_cumplen_normas = synthetic_data[filtro_edad]

#Definir los porcentajes x, y, y z que sumen 100 y representen la proporción de registros que
# tendrán los valores 1, 0 y nulo respectivamente:
x = 85  # Porcentaje de registros que tendrán el valor 1
y = 10  # Porcentaje de registros que tendrán el valor 0
z = 100 - x - y  # Porcentaje de registros que tendrán el valor nulo


# Obtener una muestra aleatoria de los registros que cumplen las normas para asignarles el valor 1:
num_registros = len(registros_cumplen_normas)
num_x = int(num_registros * x / 100)
num_y = int(num_registros * y / 100)

# Crear la columna 'T.1.' y asignar los valores correspondientes:
synthetic_data['T.4.'] = None  # Inicializar la columna con valores nulos

# Asignar el valor 1 a un porcentaje x de registros aleatorios
registros_x = registros_cumplen_normas.sample(num_x)
synthetic_data.loc[registros_x.index, 'T.4.'] = 1

# Asignar el valor 0 a un porcentaje y de registros aleatorios
registros_y = registros_cumplen_normas.drop(registros_x.index).sample(num_y)
synthetic_data.loc[registros_y.index, 'T.4.'] = 0

## T.5. Ayuda al desempleo

In [ ]:
#Filtrar el dataframe para obtener los registros que cumplen las condiciones especificadas:
filtro_desempleo = (synthetic_data['Sector económico'] == 'Parados') 
registros_cumplen_normas = synthetic_data[filtro_desempleo]

#Definir los porcentajes x, y, y z que sumen 100 y representen la proporción de registros que
# tendrán los valores 1, 0 y nulo respectivamente:
x = 85  # Porcentaje de registros que tendrán el valor 1
y = 10  # Porcentaje de registros que tendrán el valor 0
z = 100 - x - y  # Porcentaje de registros que tendrán el valor nulo


# Obtener una muestra aleatoria de los registros que cumplen las normas para asignarles el valor 1:
num_registros = len(registros_cumplen_normas)
num_x = int(num_registros * x / 100)
num_y = int(num_registros * y / 100)

# Crear la columna 'T.1.' y asignar los valores correspondientes:
synthetic_data['T.5.'] = None  # Inicializar la columna con valores nulos

# Asignar el valor 1 a un porcentaje x de registros aleatorios
registros_x = registros_cumplen_normas.sample(num_x)
synthetic_data.loc[registros_x.index, 'T.5.'] = 1

# Asignar el valor 0 a un porcentaje y de registros aleatorios
registros_y = registros_cumplen_normas.drop(registros_x.index).sample(num_y)
synthetic_data.loc[registros_y.index, 'T.5.'] = 0

## T.6. Ayuda para incentivar el consumo

In [ ]:
#Filtrar el dataframe para obtener los registros que cumplen las condiciones especificadas:
filtro_edad = (synthetic_data['Edad'] > 20) 
registros_cumplen_normas = synthetic_data[filtro_edad]

#Definir los porcentajes x, y, y z que sumen 100 y representen la proporción de registros que
# tendrán los valores 1, 0 y nulo respectivamente:
x = 70  # Porcentaje de registros que tendrán el valor 1
y = 25  # Porcentaje de registros que tendrán el valor 0
z = 100 - x - y  # Porcentaje de registros que tendrán el valor nulo


# Obtener una muestra aleatoria de los registros que cumplen las normas para asignarles el valor 1:
num_registros = len(registros_cumplen_normas)
num_x = int(num_registros * x / 100)
num_y = int(num_registros * y / 100)

# Crear la columna 'T.1.' y asignar los valores correspondientes:
synthetic_data['T.6.'] = None  # Inicializar la columna con valores nulos

# Asignar el valor 1 a un porcentaje x de registros aleatorios
registros_x = registros_cumplen_normas.sample(num_x)
synthetic_data.loc[registros_x.index, 'T.6.'] = 1

# Asignar el valor 0 a un porcentaje y de registros aleatorios
registros_y = registros_cumplen_normas.drop(registros_x.index).sample(num_y)
synthetic_data.loc[registros_y.index, 'T.6.'] = 0

## T.7. Ayuda para personas en que viven en zonas rurales

In [ ]:
## Filtrar el dataframe para obtener los registros que cumplen las condiciones especificadas:
filtro_edad = (synthetic_data['Edad'] > 20) 
filtro_ccaa = synthetic_data['Comunidad Autónoma'].isin(['La Rioja', 'Aragón','Castilla-La Mancha', 'Andalucía', 'Islas Baleares','Asturias', 'Región de Murcia', 'Galicia','Ceuta', 'Castilla y León', 'Melilla', 'Canarias', 'Cantabria','Extremadura', 'Comunidad Foral de Navarra'])
registros_cumplen_normas = synthetic_data[filtro_edad & filtro_ccaa]

#Definir los porcentajes x, y, y z que sumen 100 y representen la proporción de registros que
# tendrán los valores 1, 0 y nulo respectivamente:
x = 80  # Porcentaje de registros que tendrán el valor 1
y = 10  # Porcentaje de registros que tendrán el valor 0
z = 100 - x - y  # Porcentaje de registros que tendrán el valor nulo


# Obtener una muestra aleatoria de los registros que cumplen las normas para asignarles el valor 1:
num_registros = len(registros_cumplen_normas)
num_x = int(num_registros * x / 100)
num_y = int(num_registros * y / 100)

# Crear la columna 'T.1.' y asignar los valores correspondientes:
synthetic_data['T.7.'] = None  # Inicializar la columna con valores nulos

# Asignar el valor 1 a un porcentaje x de registros aleatorios
registros_x = registros_cumplen_normas.sample(num_x)
synthetic_data.loc[registros_x.index, 'T.7.'] = 1

# Asignar el valor 0 a un porcentaje y de registros aleatorios
registros_y = registros_cumplen_normas.drop(registros_x.index).sample(num_y)
synthetic_data.loc[registros_y.index, 'T.7.'] = 0

In [ ]:
synthetic_data.info()

In [ ]:
synthetic_data#.to_excel('C:/Users/amidonga/Documents/TFG/Datos_Sinteticos_CON_TRAMITES.xlsx',index=False)